A VQ-VAE is basically a model that learns a compressed latent space along with a codebook that maps discrete values into that latent space, and both the codebook and the latent space are trained at the same time. The key hyperparameters are basically how many discrete codes you have and the dimension of the vectors in the codebook.
